In [15]:
import os
import pandas as pd
import rasterio
import geopandas as gpd
import numpy as np
from rasterstats import zonal_stats
from shapely.geometry import Point, box
from shapely.wkt import loads
from shapely.geometry import Point
import shapely as sp

pd.set_option('display.max_columns', None)

# Check if the block group values measure up to the county values

In [16]:
suitability_county = pd.read_csv("/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Solar NIMBY Final/Solar-NIMBY/regression/cleaned data/technoecon_suitability_scores.csv", dtype={"GEOID": str})
suitability_block_group = pd.read_csv("/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Solar NIMBY Final/Solar-NIMBY/regression/cleaned data bg/block_group_suitability_scores.csv", dtype={"GEOID": str})
FIPS = pd.read_csv("/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Solar NIMBY Final/Solar-NIMBY/data cleaning/data/FIPS.csv", dtype={"FIPS State": str, "FIPS County": str})

In [17]:
FIPS

,State,County Name,FIPS State,FIPS County
0,Alabama,Autauga,01,001
1,Alabama,Baldwin,01,003
2,Alabama,Barbour,01,005
3,Alabama,Bibb,01,007
4,Alabama,Blount,01,009
...,...,...,...,...
3106,Wyoming,Sweetwater,56,037
3107,Wyoming,Teton,56,039
3108,Wyoming,Uinta,56,041
3109,Wyoming,Washakie,56,043


In [18]:
# Group and mean the values based of state county pairs for the block group level data
suitability_block_group_county = suitability_block_group.groupby(["State", "County Name"]).mean().reset_index().drop(columns=['TRACTCE', 'BLKGRPCE', "GEOID"])
suitability_block_group_county.merge(FIPS, left_on=["State", "County Name"], right_on=["State", "County Name"])

,State,County Name,GHI,Protected_Land,Habitat,Slope,Population_Density,Distance_to_Substation,Land_Cover,FIPS State,FIPS County
0,Alabama,Autauga,20.000000,99.760277,72.330639,67.949131,47.076100,74.958475,65.717212,01,001
1,Alabama,Baldwin,20.000000,88.190571,57.444079,93.217710,37.586200,59.726930,64.091507,01,003
2,Alabama,Barbour,20.000000,88.814697,36.475335,69.622183,74.815864,50.000000,59.060308,01,005
3,Alabama,Bibb,19.778329,95.359727,31.291031,49.661283,81.700240,50.419766,58.219259,01,007
4,Alabama,Blount,15.000000,99.731206,35.962092,37.856508,83.926569,62.055545,63.759352,01,009
...,...,...,...,...,...,...,...,...,...,...,...
3100,Wyoming,Sweetwater,19.984298,85.008186,59.118407,68.144035,45.251653,83.312911,81.629832,56,037
3101,Wyoming,Teton,14.439494,54.788482,25.194827,49.751728,61.292787,50.000000,67.217253,56,039
3102,Wyoming,Uinta,19.585375,83.886397,45.117874,61.747788,54.900498,50.407040,82.689538,56,041
3103,Wyoming,Washakie,18.502697,82.211098,45.345751,81.066996,48.707968,50.169588,81.189811,56,043


In [19]:
# Subtract the block group level data from the county level data on State County Name
suitability_county = suitability_county.merge(suitability_block_group_county, left_on=["State", "County Name"], right_on=["State", "County Name"], suffixes=('_county', '_block_group'))
suitability_county["suitability_score_diff"] = suitability_county["suitability_score_county"] - suitability_county["suitability_score_block_group"]
suitability_county = suitability_county.drop(columns=['suitability_score_county', 'suitability_score_block_group'])

KeyError: 'suitability_score_county'